In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import re
import os
import time
import string
import unicodedata
from pickle import load
from string import digits

In [ ]:
import tensorflow as tf

# 1. Preprocess Dataset

In [ ]:
def preprocess_sentence(sentence):
    num_digits = str.maketrans('', '', digits)
    exclude = set(string.punctuation)
    
    sentence = sentence.lower()
    sentence = re.sub(" +", " ", sentence)
    sentence = re.sub("'", "", sentence)
    sentence = ''.join(ch for ch in sentence if ch not in exclude)
    sentence = sentence.translate(num_digits)
    sentence = re.sub("[२३०८१५७९४६]", "", sentence)
    sentence = sentence.strip()
    sentence= re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = sentence.rstrip().strip()
    sentence=  'start_ ' + sentence + ' _end'
    
    return sentence

In [ ]:
max_target_length = 23
max_source_length = 9

BATCH_SIZE = 64
embedding_dim = 256
units = 1024

# 2. Load Tokenizer

In [ ]:
with open('source_tokenizer.pickle', 'rb') as handle:
    source_sentence_tokenizer = load(handle)
    
with open('target_tokenizer.pickle', 'rb') as handle:
    target_sentence_tokenizer = load(handle)

In [ ]:
vocab_inp_size = len(source_sentence_tokenizer.word_index) + 1
vocab_tar_size = len(target_sentence_tokenizer.word_index) + 1

# 3. Load Encoder-Decoder Model and Weights
### Model

In [ ]:
encoder = tf.keras.models.load_model('encoder')
decoder = tf.keras.models.load_model('decoder')

### Weights

In [ ]:
encoder.load_weights('./encoder_weights/encoder')

In [ ]:
decoder.load_weights('./decoder_weights/decoder')

# 4. Define the Optimizer and loss function

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = 'none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype = loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

# 5. Translate Input to generate Output

In [ ]:
def evaluate(sentence):
    try:
        flag = True
        sentence = preprocess_sentence(sentence)
        inputs = [source_sentence_tokenizer.word_index[i] for i in sentence.split(' ')]
        inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen = max_source_length, padding = 'post')
        inputs = tf.convert_to_tensor(inputs)
        result = ''
        hidden = [tf.zeros((1, units))]

        enc_out, enc_hidden = encoder([inputs, hidden])
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_sentence_tokenizer.word_index['start_']], 0)

        for t in range(max_target_length):
            predictions, dec_hidden, attention_weights = decoder([dec_input, dec_hidden, enc_out])
            attention_weights = tf.reshape(attention_weights, (-1, ))
            predicted_id = tf.argmax(predictions[0]).numpy()
            result += target_sentence_tokenizer.index_word[predicted_id] + ' '
            if target_sentence_tokenizer.index_word[predicted_id] == '_end':
                return result, flag

            dec_input = tf.expand_dims([predicted_id], 0)
        return result, flag
    
    except KeyError:
        flag = False
        return "Sorry we didn't find any expected translation for your entered word/sentence", flag

In [ ]:
def translate(sentence):
    result, flag = evaluate(sentence)
    if flag:
        print('Predicted translation: {}'.format(result[:-5]))
    else:
        print(result)

In [ ]:
translate(u'I am going to work.')

In [ ]:
translate(u'You need to work smart.')

In [ ]:
translate(u'Hello')

In [ ]:
translate(u'I live in Bangalore')

In [ ]:
translate(u'it fires whenever the animal gets near')

In [ ]:
translate(u'Human contracted itself blind, malignant.')

In [ ]:
translate(u'They are talking about music.')

In [ ]:
translate(u'That student loves History')

In [ ]:
translate(u'English is a beautiful language')

In [ ]:
translate(u'But modernization actually brought communication')